#Read data from processed dataframe

In [106]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
home_dir = "/content/drive/My Drive/Hackaton UNIGOTO 28.11/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [224]:
df_copy = pd.read_csv(home_dir+'data_cleaned4_combined.csv')

#Imitation user input

In [220]:
# @title
top_n = 10 # @param {type:"slider", min:0, max:20, step:1}
import ipywidgets as widgets
from IPython.display import display
import numpy as np

city = ''
education_form = ''
education_status = ''

random_numbers = [''] + df_copy['city'].value_counts().index.values.tolist()
Dropdown = widgets.Dropdown(
options=random_numbers,
description='Please, choose a city from the list',
  )
output = widgets.Output()

def onchange(change):
  global city
  city = change['new']

Dropdown.observe(onchange, names='value')
display(Dropdown)

about = str(input("Напишите о себе: "))
activities = str(input("Напишите о своих увлечениях: "))
books = str(input("Укажите ваши любимые книги: "))
games = str(input("Укажите ваши любимые игры: "))
interests = str(input("Напишите о своих интересах: "))



random_numbers = [''] + df_copy['education_form'].value_counts().index.values.tolist()
Dropdown = widgets.Dropdown(
options=random_numbers,
description='Please, choose a education_form from the list',
  )
output = widgets.Output()

def onchange(change):
  global education_form
  education_form = change['new']

Dropdown.observe(onchange, names='value')
display(Dropdown)

random_numbers = [''] + df_copy['education_status'].value_counts().index.values.tolist()
Dropdown = widgets.Dropdown(
options=random_numbers,
description='Please, choose a education_status from the list',
  )
output = widgets.Output()

def onchange(change):
  global education_status
  education_status = change['new']

Dropdown.observe(onchange, names='value')
display(Dropdown)



Dropdown(description='Please, choose a city from the list', options=('', 'Москва', 'Санкт-Петербург', 'Екатери…

Напишите о себе: программирование
Напишите о своих увлечениях: программирование
Укажите ваши любимые книги: Грокаем алгоритмы
Укажите ваши любимые игры: fifa
Напишите о своих интересах: программирование


Dropdown(description='Please, choose a education_form from the list', options=('', 'Очное отделение', 'Заочное…

Dropdown(description='Please, choose a education_status from the list', options=('', 'Выпускник', 'Студент', '…

In [221]:
user_input = [city, about, activities, books, games, interests, education_form, education_status]
user_input

['',
 'программирование',
 'программирование',
 'Грокаем алгоритмы',
 'fifa',
 'программирование',
 '',
 'Студент']

#TF-IDF recommendations

In [225]:
df_copy.fillna("", inplace=True)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [75]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 584.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=433b3d62109c1d6be86aa5cb34237fc93758f667d7f1b8c1a138f2ec2e2fed00
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [76]:
import re

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

import pymorphy2

In [77]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [78]:
stopwords_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [79]:
def clean_text(text: str) -> str:
    text = re.sub(r'((http|ftp)\S+)|[^a-zа-яё\s/-]|(-{2,})', '', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_stopwords(tokens: list, stopwords=None) -> list:
    if not stopwords:
        return tokens

    stopwords = set(stopwords)
    tokens = [tok for tok in tokens if tok not in stopwords and len(tok)>2]

    return tokens

def text_preparation(text: str) -> str:
    text = clean_text(text)
    tokens = word_tokenize(text, language='russian')
    tokens = remove_stopwords(tokens, stopwords_ru)
    tokens = [morph.parse(tok)[0].normal_form for tok in tokens]

    return ' '.join(tokens)

In [226]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x, stop_words=stopwords_ru, lowercase=False)

In [227]:
def get_recommendations():
    '''Функция для получения рекомендаций на основе введенных пользователем данных
    '''
    if city != '':
      df = df_copy[(df_copy['city'] == city)]
    else:
      df = df_copy
    if education_form != '':
      df = df[(df['education_form'] == education_form)]
    else:
      df = df


    #user_input = city + ' ' + about + ' ' + activities + ' ' + books + ' ' + games + ' ' + interests
    user_input = text_preparation(about) + text_preparation(activities) + text_preparation(books) + text_preparation(games) + text_preparation(interests)
    user_tfidf = tfidf.fit_transform([user_input]+df['combined'].tolist())

    # вычисление косинусного сходства между введенными данными и всеми пользователями
    cosine_sim_user = linear_kernel(user_tfidf[0], user_tfidf).flatten()
    print(cosine_sim_user.argsort(axis=0)[-1:-top_n-1:-1])
    # получение индекса пользователя с наибольшим косинусным сходством
    similar_user_indices = cosine_sim_user[1:].argsort(axis=0)[-1:-top_n-1:-1]
    # получение рекомендаций университета и факультета на основе наиболее похожего пользователя
    recommendations = df.loc[similar_user_indices, ['faculty_name', 'university_name']]

    return recommendations

In [229]:
get_recommendations()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[    0 48679 43055 20220 19326  5100 12049 48235  1391 32357]


,faculty_name,university_name
48678,Электротехнический,Филиал ЮУрГУ (НИУ)
43054,"Институт филологии, истории и востоковедения",СахГУ (бывш. ЮСГПИ)
20219,Факультет компьютерных технологий в бизнесе,РАНХиГС при Президенте РФ (бывш. АНХ при Прави...
19325,Музыкальный,ИТА ЮФУ (бывш. ТТИ ЮФУ)
5099,Биоинженерии и биоинформатики,МГУ
12048,Исторический,УрФУ им. первого Президента России Б. Н. Ельцина
48234,Геофизический факультет,РГГРУ (МГРИ) им. Орджоникидзе
1390,Юридический,СПбГУ
56168,Маркетинг,ДФ РАНХиГС (бывш. ВВАГС ДФ)
6236,Высшая школа бизнеса,МГУ


#SentenceTransformerModel Bert (ru version)

In [97]:
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=909f29c30efc8670ffc29509013aa1eca1f3ad784fde4241f65bcfcdf0c44890
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load pre-trained BERT model
model = SentenceTransformer('DeepPavlov/rubert-base-cased')

# Compute embeddings for the introduced text
query = about + ' ' + activities + ' ' + books + ' ' + games + ' ' + interests
query = text_preparation(query)
query_embedding = model.encode(query, convert_to_tensor=True)

# Compute embeddings for the text column
text_embeddings = model.encode(df_copy['combined'].tolist(), convert_to_tensor=True)

# Compute similarity scores
cosine_scores = util.pytorch_cos_sim(query_embedding, text_embeddings)

# Add the similarity scores to the DataFrame
df_copy['similarity'] = cosine_scores.tolist()[0]

df_copy = df_copy.sort_values(by='similarity', ascending=False)

In [217]:
import torch

text_embeddings = torch.load(home_dir+'text_embeddings_tensor.pt')

model = SentenceTransformer('DeepPavlov/rubert-base-cased')

query = about + ' ' + activities + ' ' + books + ' ' + games + ' ' + interests

query = text_preparation(query)
query_embedding = model.encode(query, convert_to_tensor=True)
query_embedding = query_embedding.type(torch.float64)

# Compute similarity scores
cosine_scores = util.pytorch_cos_sim(query_embedding, text_embeddings)

df_copy['similarity'] = cosine_scores.tolist()[0]

df_copy = df_copy.sort_values(by='similarity', ascending=False)

if city != '':
  df = df_copy[(df_copy['city'] == city)]
else:
  df = df_copy
if education_form != '':
  df = df[(df['education_form'] == education_form)]
else:
  df = df
df[['university_name', 'faculty_name']].head(top_n)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,university_name,faculty_name
62206,ПКС (филиал РГУТиС),Фармацевтический факультет
28798,ТвГУ,Факультет прикладной математики и кибернетики
22504,ПГУ,Факультет вычислительной техники (Политехничес...
15191,УГАТУ,Информатики и робототехники
8631,МГУ,Факультет экономики и управления


#WORD2VEC recommendations

In [112]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# Загружаем предобученную модель Word2Vec
word2vec_model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [117]:
def text_to_vector(text):
    vector = np.zeros(word2vec_model.vector_size)
    words = word_tokenize(text)
    for word in words:
        if word in word2vec_model.key_to_index:
            vector += word2vec_model[word]
    return vector

user_input = text_preparation(about) + ' ' + text_preparation(activities) + ' ' + text_preparation(books) + ' ' + text_preparation(games) + ' ' + text_preparation(interests)

# Преобразуем текстовые данные в векторы Word2Vec
def get_word2vec_recommendations():
  text_vectors = df_copy['combined'].apply(lambda x: text_to_vector(x))
  text_vectors = np.stack(text_vectors.values)
  user_vectors = text_to_vector(user_input)
  user_vectors = user_vectors.reshape(1, -1)
  #user_vectors = np.stack(user_vectors.values)
  # вычисление косинусного сходства между пользователями на основе векторов Word2Vec признаков
  #cosine_sim_word2vec = linear_kernel(text_vectors, text_vectors)

  cosine_sim_user_word2vec = linear_kernel(user_vectors, text_vectors)

  # получение индекса пользователя с наибольшим косинусным сходством
  similar_user_indices = cosine_sim_user_word2vec.argsort(axis=1)[0, :-top_n-1:-1]

  # получение рекомендаций университета и факультета на основе наиболее похожего пользователя
  w2v_recommendations = df_copy.loc[similar_user_indices, ['faculty_name', 'university_name']]

  return w2v_recommendations


In [118]:
get_word2vec_recommendations()

,faculty_name,university_name
62628,Экономический факультет,СФ БИЭИ (бывш. БИИММАП)
20885,Институт химии (ИнХим),ТюмГУ
20883,Физико-технический институт (ФТИ),ТюмГУ
20882,Институт дополнительного профессионального обр...,ТюмГУ
20881,Институт биологии (ИнБИО),ТюмГУ


#DOC2VEC recommendations

In [115]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

corpus = df_copy['combined'].apply(word_tokenize)
corpus = [
    TaggedDocument(words, [idx])
    for idx, words in enumerate(corpus)
]

model = Doc2Vec(corpus, min_count=0)
print(model.dv[0])

[-5.2308156e-03 -5.9791268e-03 -9.8807542e-03  8.5528456e-03
  3.5661161e-03  2.6303172e-04 -9.8806275e-03 -5.1666484e-03
 -9.7179627e-03  2.0107795e-03  2.8303110e-03  4.6435557e-03
 -4.2972756e-03 -3.1457066e-03 -3.0787874e-03 -8.7219151e-03
  2.1724831e-03  9.2256228e-03 -9.5018670e-03 -3.4580862e-03
 -3.7699090e-03  2.6073826e-03 -5.6915567e-03  2.6206803e-03
  5.8025215e-03 -8.1068603e-03 -8.3297910e-03 -9.9546695e-03
  4.9330448e-03 -9.1223074e-03  5.8419635e-03  6.8002627e-03
 -6.5064002e-03 -4.5198812e-03 -1.2548614e-03  1.6463208e-03
 -1.4813376e-03 -8.5425414e-03 -3.6026132e-03  1.7316258e-03
 -2.0569193e-03 -7.2300420e-03  4.1846000e-03 -8.5743405e-03
  2.7115368e-03 -4.6137203e-03  6.4542773e-04 -2.0573472e-03
  5.4132282e-03 -8.0025708e-03 -2.1198511e-03 -9.5815660e-05
 -6.6387774e-03 -6.5261638e-03 -1.9329584e-03  8.8034747e-03
 -1.2631691e-03  3.5359799e-03 -5.7503129e-03  8.8148145e-03
  2.9154683e-03  9.2796851e-03  4.3498552e-03 -4.1995691e-03
  2.2419060e-03 -4.41245

In [119]:
user_input = text_preparation(about) + ' ' + text_preparation(activities) + ' ' + text_preparation(books) + ' ' + text_preparation(games) + ' ' + text_preparation(interests)
#user_input = [country, city, about, activities, books, games, interests, education_status]
results = model.dv.most_similar(positive=[model.infer_vector([text_preparation(user_input)])], topn=top_n+1)
results

[(43008, 0.4005265235900879),
 (19505, 0.39529097080230713),
 (62294, 0.39437955617904663),
 (52189, 0.3918240964412689),
 (38441, 0.38916948437690735),
 (40572, 0.38421157002449036)]

In [120]:
idxs = [idx for idx, score in results]
df_copy.loc[idxs, ['faculty_name', 'university_name']]

,faculty_name,university_name
43008,Институт филологии,СахГУ (бывш. ЮСГПИ)
19505,Пищевых производств,СамГТУ
62294,Юридический,КУ МВД РФ
52189,Экономика и управление на предприятии (по отра...,ЛКИТиУ (филиал) МГУТУ им. К. Г. Разумовского (...
38441,Вечерний факультет,КГЭУ
40572,Факультет экономики и управления,ВГУВТ (ВГАВТ)
